In [ ]:
from driftmind import DriftMindClient
from driftmind.generator import generate_sin_cos_tan_with_drifts
from driftmind import utils

import pandas as pd
import matplotlib.pyplot as plt

columns = ["Sin", "Cos", "Tan"]
results = {"Sin": [], "Cos": [], "Tan": []}

# 1 Get Credentials
creds = utils.load_credentials()
if creds:
    api_key = creds["DRIFTMIND_API_KEY"]
    base_url = creds["DRIFTMIND_API_URL"]
    print("API key loaded:", bool(api_key))
    print("Base URL:", base_url)
else:
    print("Failed to load DriftMind credentials.")
    exit(1)

# 2. Connect to DriftMind
client = DriftMindClient(api_key, base_url)

# 2. Create fresh forecaster (cold start)
fid = client.create_forecaster({
    "forecasterName": "Cold Start Demo",
    "features":columns,
    "inputSize": 15,
    "outputSize": 1
})["forecaster_id"]

print("Created new forecaster with ID:", fid)

# 3. Generate dataset with drifts
df = generate_sin_cos_tan_with_drifts(n=600)

# 4. Online loop: feed one point → try forecast → store outside the package

print("Starting online learning and forecasting...")
for _, row in df.iterrows():
    point = {}
    for col in columns:
        point[col] = [float(row[col])]
    client.feed_point(fid, point)

    yhat = client.forecast(fid)
    if yhat is not None:
        for var in columns:
            pred = yhat.get(var, {}).get("predictions", [])
            if pred:
                results[var].append({
                    "expected": float(row[var]),
                    "predicted": float(pred[0]),
                    "timestamp": int(row["Sequence"])
                })
                print(f"Fed Sequence={int(row['Sequence'])}, Expected {var}={float(row[var]):.3f}, Predicted {var}={yhat.get(var, {}).get('predictions', [0])[0]:.3f}, SME={abs(float(row[var])-yhat.get(var, {}).get('predictions', [0])[0]):.3f}")


# Visualize results
for var in columns:
    df_var = pd.DataFrame(results[var])
    utils.plot_actual_vs_predicted(df_var, var)

